In [1]:
import json

import bz2
import regex
from tqdm import tqdm
from scipy import sparse

In [2]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
responses = []
with bz2.BZ2File('banki_responses.json.bz2', 'r') as thefile:
    for row in tqdm(thefile):
        resp = json.loads(row)
        if not resp['rating_not_checked'] and (len(resp['text'].split()) > 0):
            responses.append(resp)

201030it [03:17, 1019.25it/s]


#  Домашнее задание по NLP # 2 [100 баллов] 
## Составление словарей для классификации по тональности
При классификации текстов или предложений по тональности необходимо использовать оценочные словари для предметной области, то есть, такие словари, в которых содержатся отрицательные и позитивные слова для какой-то предметной области. Идея подобных словарей основана на следующих наблюдениях: во-первых, для разных товаров используются разные оценочные слова (например бывает “захватывающая книга”, но не бывает “захватывающих лыж”), во-вторых, в контексте разных товаров одни и те же слова могут иметь разную окраску (слово “тормоз” в отзыве на велосипед имеет нейтральную окраску, в отзыве на компьютер – резко негативную, “пыль” в контексте пылесосов – нейтральную, в контексте кофемолок – положительную (“мелкий помол в пыль”)). Еще один пример: "теплое пиво" – это плохо, а "теплый свитер" – это хорошо.  

Составление таких словарей в ручную – трудоемкий процесс, но, к счастью, его не сложно автоматизировать, если собрать достаточно большие корпуса отзывов. В этом домашнем задании вам предстоит попробовать реализовать один их подходов к составлению оценочных словарей, основанный на статье Inducing Domain-Specific Sentiment Lexicons from Unlabeled Corpora (https://nlp.stanford.edu/pubs/hamilton2016inducing.pdf).


Данные для задания – уже знакомые вам отзывы на банки, собранные с нескольких сайтов Рунета. Отзывы могут быть как положительными (оценка 5), так и отрицательными (оценка 1).

In [4]:
responses[99]

{'city': 'г. Саратов',
 'rating_not_checked': False,
 'title': 'Карта ко вкладу',
 'num_comments': 0,
 'bank_license': 'лицензия № 880',
 'author': 'ronnichka',
 'bank_name': 'Югра',
 'datetime': '2015-06-03 20:56:57',
 'text': 'Здравствуйте! Хотела написать, что мне месяц не выдают карту ко вкладу, ссылаясь на "нам же их из Самары везут" (на секундочку 5 часов езды от нашего города). Но! Прочитала, что людям 3,5 месяцев не выдают карту, и поняла, что у меня все хорошо, пока что. И подарок мне дали, и кулер в отделении есть. Так что я, конечно, готова ждать. Правда хотелось бы не очень долго.',
 'rating_grade': 3}

## Часть 1. Обучение модели word2vec [50 баллов]

1. Разбейте всю коллекцию отзывов на предложения. Лемматизируйте все слова. 
2. Обучите по коллекции предложений word2vec
3. Приведите несколько удачных и неудачных примеров решения стандартных текстов для word2vec:
    * тест на определение ближайших слов
    * тест на аналогии (мужчина – король : женщина – королева)
    * тест на определение лишнего слова.
    
4. Постройте несколько визуализаций:
    * TSNE для топ-100 (или топ-500) слов и найдите осмысленные кластеры слов
    * задайте координаты для нового пространства следующим образом: одна  ось описывает отношение "плохо – хорошо", вторая – "медленно – быстро" и найдите координаты названий банков в этих координатах.  Более формально:
    берем вектор слова "хорошо", вычитаем из него вектор слова "плохо", получаем новый вектор, который описывает разницу между хорошими и плохими словами. Берем вектор слова "сбербанк" и умножаем его на этот новый вектор – получаем координату по первой оси. Аналогично – для второй оси. Две координаты уже можно нарисовать на плоскости.  
 


Ссылка на примеры визуализаций: https://towardsdatascience.com/game-of-thrones-word-embeddings-does-r-l-j-part-2-30290b1c0b4b

In [5]:
col_names = list(responses[0])
df = pd.DataFrame()
for col_name in col_names:
    listt = []
    for i in range(len(responses)):
        listt.append(responses[i][col_name])
    tmp = pd.DataFrame(listt,columns=[col_name])
    df = pd.concat([df,tmp], axis=1)
    del tmp

df = df[['city','title','num_comments','bank_name','text','rating_grade']]

1. Разбейте всю коллекцию отзывов на предложения. Лемматизируйте все слова. 

In [6]:
rdf = df[(df.rating_grade==1)|(df.rating_grade==5)][['title', 'bank_name','text','rating_grade']]
rdf.head()

,title,bank_name,text,rating_grade
19,Некомпетентность сотрудников банка,Сбербанк России,Открыт вклад и счет в USD. Плюс к этому есть з...,1.0
33,Навязывание страховки,Сбербанк России,Доброго времени! Вчера мне поступило смс-уведо...,1.0
76,Нарушение договора банковского вклада,АктивКапитал Банк,"05.06.2015г. около 15 часов, пришел в указанны...",1.0
85,"Невозможно оплатить отчисления на капремонт ""Т...",Сбербанк России,Для оплаты коммунальных платежей пользуюсь пла...,1.0
94,Некомпетентность сотрудника Сбербанка при пров...,Сбербанк России,В апреле этого года пришла в отделение сбербан...,1.0


In [7]:
from nltk.corpus import stopwords
from pymystem3 import Mystem

regex = re.compile("[А-Яа-я]+")
def words_only(text, regex=regex):
    return " ".join(regex.findall(text))


mystopwords = stopwords.words('russian') + ['это', 'наш' , 'тыс', 'млн', 'млрд', 'также',  'т', 'д']
def  remove_stopwords(text, mystopwords = mystopwords):
    try:
        return " ".join([token for token in text.split() if not token in mystopwords])
    except:
        return ""

    
m = Mystem()
def lemmatize(text, mystem=m):
    try:
        return [l for l in m.lemmatize(text) if l.isalpha()]
    except:
        return []

def preprocessing(text):
    words = words_only(text)
    no_stopwords = remove_stopwords(words)
    lemmas = lemmatize(no_stopwords)
    return(lemmas)

In [8]:
%time rdf['lemmas'] = rdf.text.apply(lambda x: preprocessing(x))

CPU times: user 3min 56s, sys: 6.13 s, total: 4min 2s
Wall time: 15min 17s


2. Обучите по коллекции предложений word2vec

In [9]:
from gensim.models import Word2Vec

%time model = Word2Vec(rdf.lemmas, size=100, window=3, min_count=20, workers=4)

model.save("word2v.model")
model = Word2Vec.load("word2v.model")

CPU times: user 2min 50s, sys: 968 ms, total: 2min 51s
Wall time: 1min 33s


3. Приведите несколько удачных и неудачных примеров решения стандартных текстов для word2vec:
    * тест на определение ближайших слов
    * тест на аналогии (мужчина – король : женщина – королева)
    * тест на определение лишнего слова.

In [10]:
# тест на определение ближайших слов
model.most_similar("доход")

/home/dmitry/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


[('НДФЛ', 0.6940752267837524),
 ('заработок', 0.6604068279266357),
 ('оклад', 0.5984464287757874),
 ('зарплата', 0.5706433057785034),
 ('прибыль', 0.5568016767501831),
 ('платежеспособность', 0.5380537509918213),
 ('ЗП', 0.5359150171279907),
 ('зп', 0.5316934585571289),
 ('ндфл', 0.5305932760238647),
 ('оборот', 0.5107284188270569)]

In [11]:
# тест на аналогии (мужчина – король : женщина – королева)
model.most_similar(positive=["представитель","организация"], negative=["работник"])

/home/dmitry/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


[('компания', 0.6132292747497559),
 ('агент', 0.5596864223480225),
 ('фирма', 0.5253053903579712),
 ('брокер', 0.5127830505371094),
 ('учреждение', 0.5057264566421509),
 ('агентство', 0.5031024217605591),
 ('предприятие', 0.4654889702796936),
 ('комитет', 0.44720616936683655),
 ('контора', 0.4444267153739929),
 ('департамент', 0.44066596031188965)]

In [12]:
# тест на определение лишнего слова.
model.doesnt_match("доход вклад машина депозит".split())

/home/dmitry/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  


'машина'

4. Постройте несколько визуализаций:
    * TSNE для топ-100 (или топ-500) слов и найдите осмысленные кластеры слов
    * задайте координаты для нового пространства следующим образом: одна  ось описывает отношение "плохо – хорошо", вторая – "медленно – быстро" и найдите координаты названий банков в этих координатах.  Более формально:
    берем вектор слова "хорошо", вычитаем из него вектор слова "плохо", получаем новый вектор, который описывает разницу между хорошими и плохими словами. Берем вектор слова "сбербанк" и умножаем его на этот новый вектор – получаем координату по первой оси. Аналогично – для второй оси. Две координаты уже можно нарисовать на плоскости.

In [13]:
# TSNE для топ-100 (или топ-500) слов и найдите осмысленные кластеры слов
from nltk import FreqDist

top_words = []

fd = FreqDist()
for text in tqdm(rdf.lemmas):
    fd.update(text)
for i in fd.most_common(500):
    top_words.append(i[0])
print(top_words[:15])

100%|██████████| 62100/62100 [00:18<00:00, 3340.47it/s]

['банк', 'карта', 'я', 'деньги', 'день', 'мой', 'в', 'отделение', 'который', 'кредит', 'сотрудник', 'клиент', 'счет', 'сказать', 'получать']


In [14]:
top_words = [w for w in top_words if len(w) > 4]
top_words_vec = model[top_words]

/home/dmitry/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [15]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, random_state=0)

%time top_words_tsne = tsne.fit_transform(top_words_vec)

CPU times: user 4.93 s, sys: 39.9 ms, total: 4.97 s
Wall time: 4.83 s


In [16]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=top_words_tsne[:,0],
                                    x2=top_words_tsne[:,1],
                                    names=top_words))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

Loading BokehJS ...

In [17]:
# задайте координаты для нового пространства следующим образом: 
#     одна ось описывает отношение "плохо – хорошо", вторая – "медленно – быстро" 
#     и найдите координаты названий банков в этих координатах. 
#     Более формально: 
#         берем вектор слова "хорошо", вычитаем из него вектор слова "плохо", 
#         получаем новый вектор, который описывает разницу между хорошими и плохими словами. 
#         Берем вектор слова "сбербанк" и умножаем его на этот новый вектор – получаем координату по первой оси. 
#         Аналогично – для второй оси. Две координаты уже можно нарисовать на плоскости.

In [18]:
import numpy as np

bank_list = ['ситибанк','сбербанк','росбанк','мкб','бинбанк',
            'промсвязьбанк','втб','альфабанк','райффайзенбанк','райфайзен',
            'ситибанк','совкомбанк','собинбанк','москомприватбанк','газпромбанк']
bank_list_vec = model[bank_list]

quantity = model['хорошо'] - model['плохо']
speed =  model['быстро'] - model['медленно']

bank_list_x = []
bank_list_y = []
for el in bank_list_vec:
    bank_list_x.append(np.dot(el,quantity))
    bank_list_y.append(np.dot(el,speed))
    
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=bank_list_x,
                                    x2=bank_list_y,
                                    names=bank_list))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

/home/dmitry/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/home/dmitry/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/home/dmitry/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


## Часть 2. Распространение метки [50 баллов]

Определите 5-8 позитивных слов (например, “быстрый”, “удобный”) и 5-8  негативных слов (например,“очередь”, “медленно”). Эти слова будут основной будущего оценочного словаря. Пусть позитивному классу соответствует метка 1, негативному – -1. Пометьте выбранные слова в лексическом графе соответствующими метками. Запустите любой известный вам метод распространения метки (Label Propogation) в лексическом графе. На выходе метода распространения ошибки должны быть новые слова, помеченные метками 1 и -1 – это и есть искомые оценочные слова.

Алгоритмы распространения метки устроены примерно так: пусть мы находимся в выршине, помеченном +1. С какой-то вероятностью мы переносим эту метку на соседние узлы. С меньшей вероятностью переносим ее на вершины на расстоянии два. В конце распространения метки, часть вершин оказывается помечена меткой +1, часть – -1, большая часть остается без метки.

Рекомендуемые алгоритмы распространения метки:
1. ```graphlab.label_propagation``` (```graphlab``` доступен бесплатно по образовательной лицензии)
2. ```sklearn.semi_supervised.LabelPropagation``` 
3. ```sklearn.semi_supervised.LabelSpreading```

In [ ]:
from nltk import FreqDist

top_words = []

fd = FreqDist()
for text in tqdm(rdf.lemmas):
    fd.update(text)
for i in fd.most_common(500):
    top_words.append(i[0])
print(top_words[:15])

In [19]:
# пример построения графа 

import igraph as ig
g = ig.Graph(directed=True)
for word in model.wv.vocab.keys():
    g.add_vertex(word)
    
    
    
for word in model.wv.vocab.keys() :
    node = g.vs.select(name = word).indices[0]
    similar_words = model.most_similar(word, topn=5)
    for sim in similar_words:
        word1 = sim[0]
        val  = sim[1]
        new_node = g.vs.select(name = word1).indices[0]
        g.add_edge(node, new_node, weight = val)

DeprecationWarning: To avoid name collision with the igraph project, this visualization library has been renamed to 'jgraph'. Please upgrade when convenient.